In [1]:
import os
from chesslm.lichess import LichessPGNReader, LichessEntry

DATA_DIR = os.path.expanduser("~/Data/")
FILENAME, TOTAL_GAMES = (
    # "standard_rated_2013-01", 121_332
    "standard_rated_2017-02", 10_194_939
    # "standard_rated_2019-03", 34_869_171
    # "standard_rated_2019-10", 40_440_254
)

def accept_entry(entry: LichessEntry) -> bool:
    if entry.termination != "Normal":
        return False
    return entry.ends_with_checkmate or entry.result == "1/2-1/2"

reader = LichessPGNReader(
    os.path.join(DATA_DIR, f"lichess_db_{FILENAME}.pgn.zst"),
    TOTAL_GAMES
)

In [2]:
%%time
import pyarrow as pa

tb = pa.Table.from_pylist(
    [entry.model_dump() for entry in reader.tqdm_filter(accept_entry)]
)

Retrieved 37,938 entries: 100%|██████████| 121k/121k [00:01<00:00, 80.2kit/s]


CPU times: user 1.56 s, sys: 39.8 ms, total: 1.6 s
Wall time: 1.6 s


In [3]:
%%time
import pyarrow.parquet as pq

pq.write_table(
    tb,
    os.path.join(DATA_DIR, f"{FILENAME}_filtered.parquet.gz"),
    compression="gzip"
)

CPU times: user 1.3 s, sys: 5.58 ms, total: 1.31 s
Wall time: 1.31 s
